In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from numpy import random
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from numpy import matlib
import qgrid

In [2]:
df_x = pd.read_csv('Features_X')
df_y = pd.read_csv('Y_labels')
df_y.drop(['Unnamed: 0'],axis=1, inplace = True)
df_x.drop(['Unnamed: 0'],axis=1, inplace = True)
X = pd.DataFrame.to_numpy(df_x)
Y = pd.DataFrame.to_numpy(df_y)

In [3]:
def error_logistic(Y_pred, Y_true):
    error = 0
    for ye, y in zip(Y_pred, Y_true):
        if ye != y:
            error += 1
    error = float(error)/float(np.size(Y_pred))
    
    return error

In [4]:
def redes_neuronales(neuronas, capas_ocultas):
    capas = (neuronas)
    if capas_ocultas == 2:
        capas = (neuronas,neuronas)
    elif capas_ocultas == 3:
        capas = (neuronas,neuronas,neuronas)
    elif capas_ocultas == 4:
        capas = (neuronas,neuronas,neuronas,neuronas)
    elif capas_ocultas == 5:
        capas = (neuronas,neuronas,neuronas,neuronas,neuronas)
    elif capas_ocultas == 6:
        capas = (neuronas,neuronas,neuronas,neuronas,neuronas,neuronas)        
        
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    Error = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]
        
        #Se normalizan los datos
        media = np.mean(Xtrain)
        desvia = np.std(Xtrain)
        Xtrain = preprocessing.scale(Xtrain)
        Xtest = (Xtest - np.matlib.repmat(media, Xtest.shape[0], 1))/np.matlib.repmat(desvia, Xtest.shape[0], 1)
        
        #Llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        mlp = MLPClassifier(hidden_layer_sizes=capas,activation = 'relu',max_iter=500)
        mlp.fit(Xtrain,Ytrain)
        
        #Validación con las muestras de entrenamiento
        Ytrain_pred = mlp.predict(Xtrain)

        #Validación con las muestras de test    
        Yest = mlp.predict(Xtest)
        
        Error[j] = error_logistic(Y_pred=Yest,Y_true=Ytest)
        

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred == Ytrain)
        EficienciaVal[j] = np.mean(Yest == Ytest)
        j += 1
    print("Modelo entrenado con " + str(neuronas) + " neuronas y con " + str(capas_ocultas) + " capas ocultas" )    
    
    return np.mean(EficienciaVal), np.std(EficienciaVal),np.mean(Error),np.std(Error)

In [5]:
df_types = pd.DataFrame({
    'N. de capas ocultas' : pd.Series([1,1,1,1,1,1,1,1,1,1,1,
                                       2,2,2,2,2,2,2,2,2,2,2,
                                       3,3,3,3,3,3,3,3,3,3,3,
                                       4,4,4,4,4,4,4,4,4,4,4,
                                       5,5,5,5,5,5,5,5,5,5,5]),
    
    'Neuronas por capa' : pd.Series([10,20,30,40,50,60,70,80,90,100,120,
                                    10,20,30,40,50,60,70,80,90,100,120,
                                    10,20,30,40,50,60,70,80,90,100,120,
                                    10,20,30,40,50,60,70,80,90,100,120,
                                    10,20,30,40,50,60,70,80,90,100,120])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["Error"] = ""
df_types["Std Error"] = ""
df_types.set_index(['N. de capas ocultas','Neuronas por capa'], inplace=True)

In [ ]:
i = 0
for cap,neu in df_types.index:
    eficiencia_val, ic_val,error, std_error=redes_neuronales(neuronas=neu,capas_ocultas=cap)
    df_types["Eficiencia en validacion"][i] = str(eficiencia_val) # remplazar valores
    df_types["Intervalo de confianza"][i] = str(ic_val)
    df_types["Error"][i] = str(error)
    df_types["Std Error"][i] = str(std_error)
    print(df_types)
    i=i+1
df_types

/home/andresqb198/.local/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:916: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Modelo entrenado con 10 neuronas y con 1 capas ocultas
                                      Eficiencia en validacion  \
N. de capas ocultas Neuronas por capa                            
1                   10                      0.2942522328900545   
                    20                                           
                    30                                           
                    40                                           
                    50                                           
                    60                                           
                    70                                           
                    80                                           
                    90                                           
                    100                                          
                    120                                          
2                   10                                           
                    2

In [ ]:
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget
qgrid_widget.get_changed_df()